<a href="https://colab.research.google.com/github/VishakBharadwaj94/CNN_starter/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 

add1= torch.randint(0,9,size=[6000])
add2= torch.randint(0,9,size=[6000])
torch.cat((add1[0][None],add2[0][None])).shape

torch.Size([2])

In [2]:
import torchvision


train_data = torchvision.datasets.MNIST('./',download=True)
train_data.data.shape

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


torch.Size([60000, 28, 28])

In [3]:
class Dataset:

  def __init__(self):
    self.data = train_data.data.float()
    self.targets = train_data.targets

  def __getitem__(self,idx):

    image = torch.unsqueeze(self.data[idx],0)
    target = self.targets[idx]

    return image,target
    
  def __len__(self):
    
    return len(self.targets)

    

In [4]:
train_data = Dataset()
image,target = train_data[43]
image.shape,target

(torch.Size([1, 28, 28]), tensor(9))

In [28]:
class Model(nn.Module):

  def __init__(self):

    super().__init__()
    self.conv1 = nn.Conv2d(1,5,kernel_size=5) #24
    self.pool = nn.MaxPool2d(kernel_size=2) #12
    self.conv2 = nn.Conv2d(5,10,kernel_size=3) #10
    
    #flatten 
    self.linear1 = nn.Linear(1000,100)
    self.linear2 = nn.Linear(100,10)

  def forward(self,x):

    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = x.view(x.size(0),-1)
    x = F.relu(self.linear1(x))
    x = self.linear2(x)

    return x 

In [29]:
model = Model().cuda()
optim = torch.optim.AdamW(model.parameters(),lr=0.1)
loss_fn = nn.CrossEntropyLoss()

In [30]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,shuffle=True,batch_size=256)

In [34]:
def train(model=model,device='cuda',train_loader=train_loader,optimizer=optim,epoch=10):
    model.train()
    for epoch in range(epochs):
      losses = 0.
      for data, target in train_loader:
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.cross_entropy(output, target)
          loss.backward()
          optimizer.step()
          losses+=loss
      print(f"epoch:{epoch+1} loss={loss}")
      
train()      

epoch:1 loss=2.2993009090423584
epoch:2 loss=2.315335512161255
epoch:3 loss=2.319188356399536
epoch:4 loss=2.2771990299224854
epoch:5 loss=2.3111236095428467
epoch:6 loss=2.2887978553771973
epoch:7 loss=2.309032678604126
epoch:8 loss=2.29594087600708
epoch:9 loss=2.3022537231445312
epoch:10 loss=2.2876858711242676


# IGNORE STUFF BELOW THIS LINE

In [26]:
add_sum = add1 + add2

In [5]:
from torch import nn
from torch.nn import functional as F 

class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(2,20)
    self.linear2 = nn.Linear(20,1)

  def forward(self,x1,x2):
    inp = torch.cat((x1[None],x2[None])).float()
    out = self.linear1(inp)
    out = F.relu(out)
    out = self.linear2(out)

    return out 

In [34]:
net = Net()
optim = torch.optim.AdamW(net.parameters(),lr=0.1)
criterion = nn.MSELoss()

In [35]:
for i in range(len(add1)):

  out = net(add1[i],add2[i])
  loss = criterion(out,add_sum[i].float())
  optim.zero_grad()
  loss.backward()
  optim.step()

  if i%500==0: print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(9.5540, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0118, grad_fn=<MseLossBackward>)
tensor(0.0004, grad_fn=<MseLossBackward>)
tensor(0.3194, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(3.0294, grad_fn=<MseLossBackward>)
tensor(0.0036, grad_fn=<MseLossBackward>)
tensor(0.0049, grad_fn=<MseLossBackward>)
tensor(0.0087, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)


In [44]:
x1 = torch.tensor(5)
x2 = torch.tensor(12)

In [45]:
net(x1,x2)

tensor([17.0277], grad_fn=<AddBackward0>)